<a href="https://colab.research.google.com/github/ronenbendavid/IDC_NLP/blob/master/Asi_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3
Training a neural named entity recognition (NER) tagger 

In [1]:
import torch
import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('version: {}, device: {}'.format(torch.__version__, device))

version: 1.5.1+cu101, device: cuda


In this assignment you are required to build a full training and testing pipeline for a neural sequentail tagger for named entities, using LSTM.

The dataset that you will be working on is called ReCoNLL 2003, which is a corrected version of the CoNLL 2003 dataset: https://www.clips.uantwerpen.be/conll2003/ner/

[Train data](https://drive.google.com/file/d/1hG66e_OoezzeVKho1w7ysyAx4yp0ShDz/view?usp=sharing)

[Dev data](https://drive.google.com/file/d/1EAF-VygYowU1XknZhvzMi2CID65I127L/view?usp=sharing)

[Test data](https://drive.google.com/file/d/16gug5wWnf06JdcBXQbcICOZGZypgr4Iu/view?usp=sharing)

As you can see, the annotated texts are labeled according to the IOB annotation scheme, for 3 entity types: Person, Organization, Location.

**Task 1:** Write a funtion for reading the data from a single file (of the ones that are provided above). The function recieves a filepath and then it encodes every sentence individually using a pair of lists, one list contains the words and one list contains the tags. Each list pair will be added to a general list (data), which will be returned back from the function.

In [2]:
import requests
import re

def read_data(filepath):
    data = []

    result = re.compile(".*drive.google.com/file/d/([^/]*)/.*").match(filepath)
    if result:
      filepath = 'https://docs.google.com/uc?export=download&id={}'.format(result.group(1))
    print(filepath)

    response = requests.get(filepath)
    words = []
    tags = []

    for line in response.text.split('\n'):
        if not line:
            if len(words) > 0:
                data.append((words, tags))
            words = []
            tags = []
        else:
            line = line.strip().split()
            words.append(line[0].lower())
            tags.append(line[1])

    return data

train = read_data('https://drive.google.com/file/d/1hG66e_OoezzeVKho1w7ysyAx4yp0ShDz/view?usp=sharing')
dev = read_data('https://drive.google.com/file/d/1EAF-VygYowU1XknZhvzMi2CID65I127L/view?usp=sharing')
test = read_data('https://drive.google.com/file/d/16gug5wWnf06JdcBXQbcICOZGZypgr4Iu/view?usp=sharing')


https://docs.google.com/uc?export=download&id=1hG66e_OoezzeVKho1w7ysyAx4yp0ShDz
https://docs.google.com/uc?export=download&id=1EAF-VygYowU1XknZhvzMi2CID65I127L
https://docs.google.com/uc?export=download&id=16gug5wWnf06JdcBXQbcICOZGZypgr4Iu


The following Vocab class can be served as a dictionary that maps words and tags into Ids. The UNK_TOKEN should be used for words that are not part of the training data.

In [3]:
UNK_TOKEN = 0

class Vocab:
    def __init__(self):
        self.word2id = {"__unk__": UNK_TOKEN}
        self.id2word = {UNK_TOKEN: "__unk__"}
        self.n_words = 1
        
        self.tag2id = {"O":0, "B-PER":1, "I-PER": 2, "B-LOC": 3, "I-LOC": 4, "B-ORG": 5, "I-ORG": 6}
        self.id2tag = {0:"O", 1:"B-PER", 2:"I-PER", 3:"B-LOC", 4:"I-LOC", 5:"B-ORG", 6:"I-ORG"}
        
    def index_words(self, words):
      word_indexes = [self.index_word(w) for w in words]
      return word_indexes

    def index_tags(self, tags):
      tag_indexes = [self.tag2id[t] for t in tags]
      return tag_indexes
    
    def index_word(self, w):
        if w not in self.word2id:
            self.word2id[w] = self.n_words
            self.id2word[self.n_words] = w
            self.n_words += 1
        return self.word2id[w]
            

**Task 2:** Write a function prepare_data that takes one of the [train, dev, test] and the Vocab instance, for converting each pair of (words,tags) to a pair of indexes. Each pair should be added to data_sequences, which will be returned back from the function.

In [4]:
vocab = Vocab()

def prepare_data(data, vocab):
    data_sequences = []
    for words, tags in data:
      iwords = vocab.index_words(words)
      itags = vocab.index_tags(tags)
      data_sequences.append((iwords, itags))

    return data_sequences, vocab

train_sequences, vocab = prepare_data(train, vocab)
dev_sequences, vocab = prepare_data(dev, vocab)
test_sequences, vocab = prepare_data(test, vocab)

**Task 3:** Write NERNet, a PyTorch Module for labeling words with NER tags. 

*input_size:* the size of the vocabulary

*embedding_size:* the size of the embeddings

*hidden_size:* the LSTM hidden size

*output_size:* the number tags we are predicting for

*n_layers:* the number of layers we want to use in LSTM

*directions:* could 1 or 2, indicating unidirectional or bidirectional LSTM, respectively

The input for your forward function should be a single sentence tensor.

In [5]:
class NERNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers, bidirectional, embedding_dim=None, embedding=None):
        super(NERNet, self).__init__()

        in_features = hidden_size
        if bidirectional:
            in_features = hidden_size * 2

        if embedding:
            self.embedding = embedding
        else:
            self.embedding = nn.Embedding(num_embeddings=input_size, embedding_dim=embedding_dim)

        self.lstm = nn.LSTM(input_size=self.embedding.embedding_dim, hidden_size=hidden_size, num_layers=n_layers, bidirectional=bidirectional, batch_first=True)
        self.out = nn.Linear(in_features=in_features, out_features=output_size)

    def forward(self, input_sentence):
        embedded = self.embedding(input_sentence)
        output, _ = self.lstm(embedded)

        # shape for the linear layer (batch*seq, num_tags)
        output = output.view(-1, output.shape[2])
        return self.out(output)

**Task 4:** write a training loop, which takes a model (instance of NERNet) and number of epochs to train on. The loss is always CrossEntropyLoss and the optimizer is always Adam.

In [6]:
import torch.optim as optim

def train_loop(model, n_epochs):
    # Loss function
    criterion = nn.CrossEntropyLoss()

    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    e_loss = 0
    for e in range(1, n_epochs + 1):

        e_loss = 0
        for counter, (sequence, labels) in enumerate(train_sequences):

            sequence_tensor = torch.LongTensor(sequence).to(device).view(1, -1)
            labels_tensor = torch.LongTensor(labels).to(device)

            # compute model output and loss
            output_batch = model(sequence_tensor)
            sentence_loss = criterion(output_batch, labels_tensor)

            optimizer.zero_grad()
            sentence_loss.backward()

            # updating weights
            optimizer.step()

            # averaging total loss
            sentence_loss = sentence_loss.item() / len(sequence)
            e_loss += sentence_loss

        e_loss = e_loss / len(train_sequences)
        # print('Epoch %d/%d, Current Loss = %.4f' % (e, n_epochs, e_loss))
    
    return e_loss



**Task 5:** write an evaluation loop on a trained model, using the dev and test datasets. This function print the true positive rate (TPR), also known as Recall and the opposite to false positive rate (FPR), also known as precision, of each label seperately (7 labels in total), and for all the 6 labels (except O) together. The caption argument for the function should be served for printing, so that when you print include it as a prefix.

In [7]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report

def evaluate_dataset(model, dataset):
    y_true = np.array([])
    y_pred = np.array([])

    for counter, (sequence, labels) in enumerate(dataset):
        sequence_tensor = torch.LongTensor(sequence).to(device).view(1, -1)
        output_labels = model(sequence_tensor)

        labels = np.array(labels)
        pred_labels = torch.argmax(output_labels, dim=1).detach().cpu().numpy()

        y_true = np.append(y_true, labels)
        y_pred = np.append(y_pred, pred_labels)

    report = classification_report(y_true, y_pred, target_names=list(vocab.tag2id.keys()), digits=4, output_dict=True, zero_division=False)
    precision = [report[label]['precision'] for label in vocab.tag2id.keys()]
    recall = [report[label]['recall'] for label in vocab.tag2id.keys()]

    report = classification_report(y_true > 0, y_pred > 0, target_names=['O', 'OTHER'], digits=4, output_dict=True)
    precision.append(report['OTHER']['precision'])
    recall.append(report['OTHER']['recall'])

    rows = ['precision', 'recall']
    columns = list(vocab.tag2id.keys())
    columns.append('Not O')

    df = pd.DataFrame(np.array([precision, recall]), index=rows, columns=columns)
    print(df)


def evaluate(model):
    print("Dev")
    evaluate_dataset(model, dev_sequences)
    print("\nTest")
    evaluate_dataset(model, test_sequences)
  

In [8]:
# model = NERNet(vocab.n_words, 500, len(vocab.tag2id), 1, False, embedding_dim=300).to(device)
# caption = "hidden_size: {}, n_layers: {}, bidirectional: {}".format(500, 1, False)

# loss = train_loop(model, 10)

In [9]:
# pd.options.display.max_columns = None
# pd.options.display.width = 1000

# print("\n----\n%s, Epochs: %d, Loss: %.4f" % (caption, 10, e_loss))

# evaluate(model)

**Task 6:** Train and evaluate a few models, all with embedding_size=300, and with the following hyper parameters (you may use that as captions for the models as well):

Model 1: (hidden_size: 500, n_layers: 1, directions: 1)

Model 2: (hidden_size: 500, n_layers: 2, directions: 1)

Model 3: (hidden_size: 500, n_layers: 3, directions: 1)

Model 4: (hidden_size: 500, n_layers: 1, directions: 2)

Model 5: (hidden_size: 500, n_layers: 2, directions: 2)

Model 6: (hidden_size: 500, n_layers: 3, directions: 2)

Model 4: (hidden_size: 800, n_layers: 1, directions: 2)

Model 5: (hidden_size: 800, n_layers: 2, directions: 2)

Model 6: (hidden_size: 800, n_layers: 3, directions: 2)

In [10]:
def train_and_evaluate(hidden_size, n_layers, bidirectional, embedding=None, embedding_dim=300, n_epochs=10):
    model = NERNet(vocab.n_words, hidden_size, len(vocab.tag2id), n_layers, bidirectional, embedding=embedding, embedding_dim=embedding_dim).to(device)
    caption = "hidden_size: {}, n_layers: {}, bidirectional: {}".format(hidden_size, n_layers, bidirectional)

    loss = train_loop(model, n_epochs)

    print("\n----\n%s, Epochs: %d, Loss: %.4f" % (caption, n_epochs, loss))
    evaluate(model)


In [ ]:
pd.options.display.max_columns = None
pd.options.display.width = 1000

train_and_evaluate(500, 1, False, n_epochs=10)
train_and_evaluate(500, 2, False, n_epochs=10)
train_and_evaluate(500, 3, False, n_epochs=10)
train_and_evaluate(500, 1, True, n_epochs=10)
train_and_evaluate(500, 2, True, n_epochs=10)
train_and_evaluate(500, 3, True, n_epochs=10)
train_and_evaluate(800, 1, True, n_epochs=10)
train_and_evaluate(800, 2, True, n_epochs=10)
train_and_evaluate(800, 3, True, n_epochs=10)


----
hidden_size: 500, n_layers: 1, bidirectional: False, Epochs: 10, Loss: 0.0018
Dev
                  O     B-PER     I-PER     B-LOC     I-LOC     B-ORG     I-ORG     Not O
precision  0.917804  0.758621  0.882353  0.843972  0.909091  0.677852  0.826923  0.891641
recall     0.977390  0.660000  0.668790  0.650273  0.434783  0.601190  0.370690  0.680047

Test
                  O     B-PER     I-PER     B-LOC     I-LOC     B-ORG     I-ORG     Not O
precision  0.914720  0.744444  0.835443  0.837037  1.000000  0.640569  0.521739  0.853870
recall     0.971829  0.617512  0.668919  0.658892  0.490566  0.514286  0.240000  0.644988

----
hidden_size: 500, n_layers: 2, bidirectional: False, Epochs: 10, Loss: 0.0009
Dev
                  O     B-PER     I-PER     B-LOC     I-LOC     B-ORG     I-ORG     Not O
precision  0.926859  0.696335  0.891667  0.866667  0.909091  0.650685  0.690141  0.883890
recall     0.974160  0.665000  0.681529  0.710383  0.434783  0.565476  0.422414  0.719008

Test
  

**Task 6:** Download the GloVe embeddings from https://nlp.stanford.edu/projects/glove/ (use the 300-dim vectors from glove.6B.zip). Then intialize the nn.Embedding module in your NERNet with these embeddings, so that you can start your training with pre-trained vectors. Repeat Task 6 and print the results for each model.

Note: make sure that vectors are aligned with the IDs in your Vocab, in other words, make sure that for example the word with ID 0 is the first vector in the GloVe matrix of vectors that you initialize nn.Embedding with. For a dicussion on how to do that, check it this link:
https://discuss.pytorch.org/t/can-we-use-pre-trained-word-embeddings-for-weight-initialization-in-nn-embedding/1222

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np

glove_path = '/content/drive/My Drive/datasets/glove'

words_found = 0
weights_matrix = np.full(vocab.n_words, None)
with open(f'{glove_path}/glove.6B.300d.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]

        idx = vocab.word2id.get(word, None)
        if idx is None:
            continue

        weights_matrix[idx] = np.array(line[1:]).astype(np.float)
        words_found += 1

for i in range(vocab.n_words):
    if weights_matrix[i] is None:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(300,))

weights_matrix = np.stack(weights_matrix, axis=0)

print('words_found: {} out of: {}'.format(words_found, vocab.n_words))


In [ ]:
num_embeddings, embedding_dim = weights_matrix.shape
embedding = nn.Embedding(num_embeddings, embedding_dim)
embedding.weight.data.copy_(torch.from_numpy(weights_matrix))

print(embedding)

In [ ]:
pd.options.display.max_columns = None
pd.options.display.width = 1000

train_and_evaluate(500, 1, False, n_epochs=10, embedding=embedding)
train_and_evaluate(500, 2, False, n_epochs=10, embedding=embedding)
train_and_evaluate(500, 3, False, n_epochs=10, embedding=embedding)
train_and_evaluate(500, 1, True, n_epochs=10, embedding=embedding)
train_and_evaluate(500, 2, True, n_epochs=10, embedding=embedding)
train_and_evaluate(500, 3, True, n_epochs=10, embedding=embedding)
train_and_evaluate(800, 1, True, n_epochs=10, embedding=embedding)
train_and_evaluate(800, 2, True, n_epochs=10, embedding=embedding)
train_and_evaluate(800, 3, True, n_epochs=10, embedding=embedding)

**Good luck!**